In [226]:
#!/usr/bin/env python
import ConfigParser
import logging
import boto.sqs
from boto.sqs.message import Message
import boto.dynamodb
from boto.s3.connection import S3Connection
import time
import shutil
import os
from boto.s3.key import Key

In [227]:
config = ConfigParser.RawConfigParser() 

In [228]:
config.read("D:\\Cloud1\\animato_master\\configs\\worker.cfg")
#config.read(config_file)

['D:\\Cloud1\\animato_master\\configs\\worker.cfg']

In [229]:
config.sections()

['AWS_CRED', 'WEB_SERVER', 'SQS', 'DYNAMODB', 'WATCHER', 'WORKER']

In [230]:
logging.basicConfig(filename='../logs/worker.log',level=logging.DEBUG)

In [231]:
def create_sqs_connection(region,access_key_id,secret_access_key,queue_name):
        """Create connection object to sqs 
        Args:
            region: aws region where sqs is present
            aws_access_key_id,aws_secret_access_key : credentials to make connection
            queue_name: name of the queue
        """
        conn = boto.sqs.connect_to_region(region,
                                      aws_access_key_id= access_key_id,
                                      aws_secret_access_key= secret_access_key
                                      )
        if conn:
            queue_obj = conn.get_queue(queue_name)
            if queue_obj:
                logging.info('returning %s queueobject '%(queue_name))
                return queue_obj
            else:
                logging.error('unable to get %s queue '%(queue_name))
                return None
        else:
            logging.error('unable to make connection to aws')
            return None

In [232]:
print config

<ConfigParser.RawConfigParser instance at 0x0000000006CC7CC8>


In [233]:
sqsObj = create_sqs_connection(config.get('AWS_CRED','REGION'),
                                   config.get('AWS_CRED','AWS_ACCESS_KEY_ID'),
                                   config.get('AWS_CRED','AWS_SECRECT_ACCESS_KEY'),
                                   config.get('SQS','QUEUE_NAME')
                                   )

In [241]:
rs = sqsObj.get_messages()
message = rs[0]
message = message.get_body()
message = message.split('|')
print message
print len(message)
customer_id = message[0]
job = message[1:]
print customer_id
print job

[u'3', u'AA']
2
3
[u'AA']


In [235]:
def create_dynamo_db_connection(region,access_key_id,secret_access_key,table_name):
        """
        Used to get table Object
        Args:
            region: aws region where sqs is present
            aws_access_key_id,aws_secret_access_key : credentials to make connection
            table_name: name of the table
        """
        conn = boto.dynamodb.connect_to_region(region,
                                                aws_access_key_id=access_key_id,
                                                aws_secret_access_key=secret_access_key
                                                )
        if conn:
            tableObj = conn.get_table(table_name)
            if tableObj:
                logging.info('returning %s tableObj '%(table_name))
                return tableObj
            else:
                logging.error('unable to create %s tableObj '%(table_name))
                return None
        else:
            logging.error('unable to make connection to aws')
            return None

In [236]:
dynamoObj = create_dynamo_db_connection(config.get('AWS_CRED','REGION'),
                                   config.get('AWS_CRED','AWS_ACCESS_KEY_ID'),
                                   config.get('AWS_CRED','AWS_SECRECT_ACCESS_KEY'),
                                   config.get('DYNAMODB','TABLE_NAME')
                                   )

In [237]:
print dynamoObj

Table(JobScheduler)


In [238]:
#Through boto add to sqs
m = Message()
m.set_body('3'+"|"+'AA')
sqsObj.write(m)

In [240]:
#Through boto add to dynamodb
data ={'job_stats':'C','try_count':0}
item = dynamoObj.new_item(hash_key = 3, 
                                  attrs= data
                                  )
item.put()

{u'ConsumedCapacityUnits': 1.0}

In [242]:
def fetch_job_from_sqs():
        rs = sqsObj.get_messages()
        if rs and rs[0]:
            message = rs[0]
            message = message.get_body()
            message = message.split('|')
            #if len(message) < 2:  #---------why is this set to false?
                #return False
            customer_id = message[0]
            global job
            job = message[1:]
            message = rs[0]
            return True
        return False

In [243]:
def check_item_exists():
        """Checks whether an item exists in DynamoDB or Not
        """
        try:
            item = dynamoObj.get_item(
            hash_key = self.customer_id)
            return item
        except:
            return None

In [244]:
def update_count_by_1():
        item = check_item_exists()
        if item:
            item['try_count'] = item['try_count']+1
            item.put()

In [245]:
def is_job_already_taken():
        item = check_item_exists()
        if item and (item['job_stats'] == 'P' and item['try_count'] <3 ):
            update_count_by_1()
            return True
        return False

In [246]:
def change_job_status(status):
        item = check_item_exists()
        if item:
            item['job_stats'] = status
            item.put()
            return True
        
        return False

In [247]:
def create_s3_connection(access_key_id,secret_access_key,bucket_name):
        conn = S3Connection(access_key_id,secret_access_key)
        if conn:
            s3obj = conn.get_bucket(bucket_name)
            if s3obj:
               logging.info('returning %s s3Obj '%(bucket_name))
               return s3obj
            else:
                logging.error('unable to create %s s3Obj '%(bucket_name))
                return None
        else:
            logging.error('unable to make connection to aws')
            return None

In [248]:
def download_files_from_s3(s3obj):
        directory = './'+self.customer_id+'/'
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory) 
        for key in s3obj.list():
            key.get_contents_to_filename(directory+'/'+key.name)

In [249]:
def process_downloaded_files():
        directory = './'+customer_id+'/'
        cmd = 'convert '+directory+'*.jpg -delay 50 -morph 10 '+ directory+'out.mpeg'
        ret = os.system(cmd)
        return ret

In [250]:
def put_video_to_s3(s3obj):
        k = Key(s3obj)
        k.key = 'myvideo.mpeg'
        filename = './'+customer_id+'/'+'out.mpeg'
        k.set_contents_from_filename(filename)
        
        shutil.rmtree('./'+customer_id+'/')
        return True

In [251]:
def process_video_job(bucket_name):
        s3obj = create_s3_connection(config.get('AWS_CRED','AWS_ACCESS_KEY_ID'),
                                   config.get('AWS_CRED','AWS_SECRECT_ACCESS_KEY'),
                                   bucket_name
                                   )
        if s3obj:
            logging.info("connected to %s S3 bucket"%(bucket_name))
            download_files_from_s3(s3obj)
            if process_downloaded_files() ==0:
                put_video_to_s3(s3obj)
                logging.info("video now available in %s bucket"%(bucket_name))
                return True
            
        else:
            logging.error("Unable to connect to %s S3 bucket"%(bucket_name))

In [252]:
def process_job():
        for j in job:
            '''
            t = job.strip().split()
            if len(t) == 2:#sleep job
                logging.debug('Sleeping for %d seconds'%(int(t[1])))
                time.sleep(int(t[1]))
            elif len(t) == 1:#video  job
                logging.debug('Started video  job')
                process_video_job(job)
            else:
                return False
            '''
            logging.debug('Started video  job')
            process_video_job(j)
            
        return True

In [253]:
print job

[u'AA']


In [254]:
def remove_job_from_sqs():
        sqsObj.delete_message(message)
        return True

In [255]:
while(True):
            if fetch_job_from_sqs() and not is_job_already_taken():
                change_job_status('P')
                if process_job():
                    change_job_status('F')
                else:
                    change_job_status('E')
                    
                remove_job_from_sqs()

AttributeError: 'list' object has no attribute 'receipt_handle'